In [5]:
import os
import torch

base_name = 'random16M_vocab32_1K'
model = base_name + ".pth"
ckpt_path = os.path.join("eval_models",model)
checkpoint = torch.load(ckpt_path,map_location = "cpu")
print(checkpoint['iter_num'])
print(checkpoint.keys())
print(checkpoint['model_args'])


1000
dict_keys(['model', 'optimizer', 'model_args', 'iter_num', 'train_loss_list', 'val_loss_list', 'best_val_loss', 'config'])
{'n_layer': 8, 'n_head': 8, 'n_embd': 512, 'block_size': 1023, 'bias': False, 'vocab_size': 32, 'dropout': 0.0}


/tmp/ipykernel_65877/3730475234.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path,map_location = "cpu")
